In [1]:
import cudf 
from pathlib import Path

In [2]:
data_path = Path.cwd().parent / 'preprocess/data'
temp_path = Path.cwd() / 'data'
if not temp_path.is_dir():
    temp_path.mkdir()

## LB

In [3]:
lb_in = data_path / 'cv'
lb_out = temp_path / 'lb'
if not lb_out.is_dir():
    lb_out.mkdir()

In [4]:
data = cudf.read_parquet((lb_in / 'test_sessions.parquet').as_posix())
data[['session', 'aid']] = data[['session', 'aid']].astype('int32')

data = data.sort_values(
    ['session', 'ts'], ascending=[True, False], ignore_index=True
)

data.drop(['ts', 'type'], axis=1, inplace=True)

data.drop_duplicates(['session', 'aid'])
data['n'] = data.groupby('session').cumcount()
data = data.loc[data.n < 20].drop('n', axis=1)

data['past'] = True
data = data.rename(columns={'aid': 'candidate'})

data.to_parquet((lb_out / 'cand.parquet').as_posix())